# Hoofdstuk 7: Chatapplicaties bouwen
## OpenAI API Snelstart

Dit notebook is aangepast van de [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) die notebooks bevat die toegang geven tot [Azure OpenAI](notebook-azure-openai.ipynb) diensten.

De Python OpenAI API werkt ook met Azure OpenAI Modellen, met een paar aanpassingen. Lees hier meer over de verschillen: [Hoe je kunt wisselen tussen OpenAI en Azure OpenAI endpoints met Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# Overzicht  
"Grote taalmodellen zijn functies die tekst naar tekst omzetten. Wanneer je een invoertekst geeft, probeert een groot taalmodel te voorspellen welke tekst er daarna zal komen"(1). Deze "quickstart"-notebook geeft gebruikers een introductie tot de belangrijkste LLM-concepten, de kernvereisten om te starten met AML, een eenvoudige introductie tot promptontwerp, en enkele korte voorbeelden van verschillende toepassingsmogelijkheden.


## Inhoudsopgave  

[Overzicht](../../../../07-building-chat-applications/python)  
[Hoe gebruik je OpenAI Service](../../../../07-building-chat-applications/python)  
[1. Je OpenAI Service aanmaken](../../../../07-building-chat-applications/python)  
[2. Installatie](../../../../07-building-chat-applications/python)    
[3. Inloggegevens](../../../../07-building-chat-applications/python)  

[Toepassingen](../../../../07-building-chat-applications/python)    
[1. Tekst samenvatten](../../../../07-building-chat-applications/python)  
[2. Tekst classificeren](../../../../07-building-chat-applications/python)  
[3. Nieuwe productnamen genereren](../../../../07-building-chat-applications/python)  
[4. Een classifier verfijnen](../../../../07-building-chat-applications/python)  

[Referenties](../../../../07-building-chat-applications/python)


### Bouw je eerste prompt  
Deze korte oefening geeft een basisintroductie voor het indienen van prompts bij een OpenAI-model voor een eenvoudige taak: "samenvatten".


**Stappen**:  
1. Installeer de OpenAI-bibliotheek in je python-omgeving  
2. Laad standaard hulplibraries en stel je gebruikelijke OpenAI-beveiligingsgegevens in voor de OpenAI Service die je hebt aangemaakt  
3. Kies een model voor je taak  
4. Maak een eenvoudige prompt voor het model  
5. Dien je verzoek in bij de model-API!


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. Het juiste model vinden  
De GPT-3.5-turbo of GPT-4 modellen kunnen natuurlijke taal begrijpen en genereren.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. Promptontwerp  

"Het bijzondere aan grote taalmodellen is dat ze, door getraind te worden om de voorspellingsfout te minimaliseren op enorme hoeveelheden tekst, uiteindelijk concepten leren die nuttig zijn voor deze voorspellingen. Bijvoorbeeld, ze leren concepten zoals"(1):

* hoe je woorden spelt
* hoe grammatica werkt
* hoe je kunt parafraseren
* hoe je vragen beantwoordt
* hoe je een gesprek voert
* hoe je in verschillende talen schrijft
* hoe je codeert
* enzovoort

#### Hoe stuur je een groot taalmodel aan  
"Van alle invoer die een groot taalmodel krijgt, is de tekstprompt verreweg het meest bepalend(1).

Grote taalmodellen kunnen op verschillende manieren worden aangespoord om output te genereren:

Instructie: Vertel het model wat je wilt
Aanvulling: Laat het model het begin van wat je wilt afmaken
Demonstratie: Laat het model zien wat je wilt, met:
Een paar voorbeelden in de prompt
Honderden of duizenden voorbeelden in een fine-tuning trainingsdataset"



#### Er zijn drie basisrichtlijnen voor het maken van prompts:

**Laat zien en vertel**. Maak duidelijk wat je wilt, door middel van instructies, voorbeelden, of een combinatie van beide. Als je wilt dat het model een lijst alfabetisch rangschikt of een alinea indeelt op sentiment, laat dan zien dat dat is wat je wilt.

**Lever goede data aan**. Als je een classifier wilt bouwen of het model een patroon wilt laten volgen, zorg dan dat er genoeg voorbeelden zijn. Controleer je voorbeelden goed — het model is meestal slim genoeg om simpele spelfouten te doorzien en je toch een antwoord te geven, maar het kan ook denken dat dit expres is en dat beïnvloedt het antwoord.

**Controleer je instellingen.** De temperatuur- en top_p-instellingen bepalen hoe voorspelbaar het model is bij het genereren van een antwoord. Als je een antwoord wilt waar maar één juist antwoord mogelijk is, zet deze dan lager. Als je juist meer variatie wilt, kun je ze hoger zetten. De grootste fout die mensen maken met deze instellingen is denken dat het "slimheid" of "creativiteit" regelaars zijn.


Bron: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Tekst Samenvatten  
#### Uitdaging  
Vat tekst samen door 'tl;dr:' aan het einde van een tekstpassage toe te voegen. Let op hoe het model in staat is om verschillende taken uit te voeren zonder extra instructies. Je kunt experimenteren met meer beschrijvende prompts dan tl;dr om het gedrag van het model aan te passen en de samenvatting die je ontvangt te personaliseren(3).  

Recent onderzoek heeft aanzienlijke vooruitgang laten zien op veel NLP-taken en benchmarks door eerst te pre-trainen op een grote hoeveelheid tekst en daarna fijn af te stemmen op een specifieke taak. Hoewel deze methode qua architectuur meestal taak-onafhankelijk is, zijn er nog steeds taak-specifieke datasets nodig met duizenden of tienduizenden voorbeelden voor de fine-tuning. Daarentegen kunnen mensen meestal een nieuwe taaltaak uitvoeren met slechts een paar voorbeelden of simpele instructies – iets waar huidige NLP-systemen nog steeds moeite mee hebben. Hier laten we zien dat het opschalen van taalmodellen de taak-onafhankelijke prestaties bij weinig voorbeelden sterk verbetert, soms zelfs tot het niveau van eerdere state-of-the-art fine-tuning methodes. 



Tl;dr


# Oefeningen voor verschillende gebruikssituaties  
1. Tekst samenvatten  
2. Tekst classificeren  
3. Nieuwe productnamen genereren


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Tekst Classificeren  
#### Uitdaging  
Classificeer items in categorieën die tijdens de inferentie worden opgegeven. In het volgende voorbeeld geven we zowel de categorieën als de te classificeren tekst in de prompt (*playground_reference).

Klantvraag: Hallo, een van de toetsen op mijn laptoptoetsenbord is onlangs kapot gegaan en ik heb een vervanging nodig:

Geclassificeerde categorie:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Genereer Nieuwe Productnamen
#### Uitdaging
Bedenk productnamen op basis van voorbeeldwoorden. In deze opdracht geven we informatie over het product waarvoor we namen gaan verzinnen. We geven ook een vergelijkbaar voorbeeld om het gewenste patroon te laten zien. Daarnaast hebben we de temperatuurwaarde hoog ingesteld om meer willekeur en creativiteit in de antwoorden te krijgen.

Productbeschrijving: Een milkshakemachine voor thuis
Zaadwoorden: snel, gezond, compact.
Productnamen: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Productbeschrijving: Een paar schoenen die op elke voetmaat passen.
Zaadwoorden: aanpasbaar, pasvorm, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Referenties  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Voorbeelden](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Best practices voor het fijn afstemmen van GPT-3 om tekst te classificeren](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Voor meer hulp  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Bijdragers
* Louis Li



---

**Disclaimer**:
Dit document is vertaald met behulp van de AI-vertalingsdienst [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u er rekening mee te houden dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet als de gezaghebbende bron worden beschouwd. Voor kritische informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
